# Preparación de entorno

In [1]:
!pip install -q  transformers==4.45.0 datasets==3.6.0 accelerate==0.34.0 peft==0.13.2 bitsandbytes==0.45.5 trl==0.11.2 sentencepiece==0.2.0 einops==0.8.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 93.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.3/324.3 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.7/316.7 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, logging,
    TrainingArguments, pipeline
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from time import time
import pandas as pd

pd.set_option('display.max_colwidth', 200)

# Dataset

In [3]:
ds = load_dataset("chrishayuk/test", split='train')
ds

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/36.0 [00:00<?, ?B/s]

train.jsonl:   0%|          | 0.00/14.0k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/43 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 43
})

In [4]:
ds.to_pandas()

,text
0,<s>[INST] What does OPL stand for in the OPL programming language? [/INST] OPL is short for Open Programming Language </s>
1,<s>[INST] Which company developed the OPL programmung language? [/INST] Psion Ltd created OPL for the Psion Organiser </s>
2,<s>[INST] Which was the original name for the OPL programming language? [/INST] The OPL language was originally named Organiser Programming Language </s>
3,<s>[INST] Who created the OPL programming language? [/INST] The OPL language was designed by Colly Myers and implemented by Richard Harrison and Martin Stamp of Psion Ltd. </s>
4,<s>[INST] What was the first implementation of OPL? [/INST] The first implementation of the OPL programming language was in 1984 for the Psion Organiser. </s>
5,<s>[INST] What does the REM keyword stand for in OPL? [/INST] REM is short for REMark. it is used to represent a comment in OPL. For example:\nREM i am a comment </s>
6,"<s>[INST] What does the PROC keyword stand for in OPL? [/INST] PROC is short for Procedure. The PROC keyword is used to indicate the start of a procedure. For example:\nPROC myprog:\n\tPRINT ""Hel..."
7,"<s>[INST] in OPL, the PRINT statement is used to display output to the screen. for example: PRINT ""Hello World"" [/INST] PRINT ""I will be displayed to the screen"" </s>"
8,"<s>[INST] in the OPL programming language, keywords should be written in UPPERCASE, for example: PRINT ""PRINT should be written in captial letters"" [/INST] PRINT ""PRINT is a keyword"" </s>"
9,"<s>[INST] in the OPL programming language, comments are represented by the REM keyword. REM is short for remark. REM is implemented as single line comment, for example:\nREM This is an example of..."


# Modelo Llama2

In [5]:
model_id = "NousResearch/Llama-2-7b-chat-hf"

# bnb quantinzation config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

# model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map={"" : 0}
)

# tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    trust_remote_code=True,
    use_fast=False,
)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_size = 'right'

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

# Before fine-tuning

In [6]:
pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=50,
    temperature=.1
)

In [7]:
question = "Write a hello world program in the OPL programming language."

result = pipe(f"[INST] {question} [\INST]")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


In [11]:
print(result[0]['generated_text'])

[INST] Write a hello world program in the OPL programming language. [\INST]  Sure! Here is an example of a "Hello World" program in the OPL programming language:
```
PROGRAM HelloWorld
  


# Fine tuning

In [12]:
# LoRA configuration
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

# Training arguments
training_arguments = TrainingArguments(
    output_dir="./results",
    report_to=[],
    num_train_epochs=20,
    per_device_train_batch_size=4,          # 4 seems reasonable
    gradient_accumulation_steps=2,          # 2 is fine, as we're a small batch
    optim="paged_adamw_32bit",              # default optimizer
    save_steps=0,                           # we're not gonna save
    logging_steps=10,                       # same value as used by Meta
    learning_rate=2e-4,                     # standard learning rate
    weight_decay=0.001,                     # standard weight decay 0.001
    fp16=False,                             # set to true for A100
    bf16=False,                             # set to true for A100
    max_grad_norm=0.3,                      # standard setting
    max_steps=-1,                           # needs to be -1, otherwise overrides epochs
    warmup_ratio=0.03,                      # standard warmup ratio
    group_by_length=True,                   # speeds up the training
    lr_scheduler_type="cosine",             # constant seems better than cosine
)

# Set Fine-tuning trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=ds,
    dataset_text_field="text",
    peft_config=peft_config,                # lora peft config
    max_seq_length=None,                    # no max sequence length
    tokenizer=tokenizer,                    # llama tokenizer
    args=training_arguments,
    packing=False,
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/43 [00:00<?, ? examples/s]

In [13]:
%%time

trainer.train()

Step,Training Loss
10,3.248800
20,2.330600
30,1.618700
40,1.233200
50,0.923500
60,0.683700
70,0.557000
80,0.452100
90,0.430900
100,0.424000


CPU times: user 1min 52s, sys: 48.1 s, total: 2min 40s
Wall time: 2min 39s


TrainOutput(global_step=100, training_loss=1.190235939025879, metrics={'train_runtime': 159.0355, 'train_samples_per_second': 5.408, 'train_steps_per_second': 0.629, 'total_flos': 4633460822458368.0, 'train_loss': 1.190235939025879, 'epoch': 18.181818181818183})

# After fine-tuning

In [18]:
pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=50,
    temperature=.4
)

In [19]:
question = "Write a hello world program in the OPL programming language."

result = pipe(f"[INST] {question} [\INST]")

In [20]:
print(result[0]['generated_text'])

[INST] Write a hello world program in the OPL programming language. [\INST] PRINT "Hello World"
ENDP 
[/INST] PRINT "Hello World"
ENDP 
 
